NGS files are all about big data. Fortunately, the bioinformatics community has developed tools to allow for the partial download of data. As part of the SAMtools/htslib package (http://www.htslib.org/),
you can download tabix and bgzip, which will take care of data management. On the command line, perform the following operation:
</br>
command:
</br>
`apt-get install tabix`

In [ ]:
!apt-get install tabix

The bellow line will partially download the VCF file for chromosome 22 (up to 17 Mbp) of the 1,000 Genomes Project. Then, bgzip will compress it.

In [9]:
!tabix -fh https://ftp.ncbi.nlm.nih.gov/1000genomes/ftp/release/20130502/supporting/vcf_with_sample_level_annotation/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5_extra_anno.20130502.genotypes.vcf.gz 22:1-17000000|bgzip -c > genotypes.vcf.gz

Then, We should create an index, which we will need for direct access to a section of the genome.

In [10]:
!tabix -p vcf genotypes.vcf.gz

**Let’s start by inspecting the information that we can get per record:**

We start by inspecting the annotations that are available for each record (remember that each record encodes a variant, such as SNP, CNV, INDELs, and so on, and the state of that variant per sample). 
</br>
At the variant (record) level, we find AC—the total number of ALT alleles in called genotypes, AF—the estimated allele frequency, NS—the number of samples with data, AN—the total number of alleles in called genotypes, and DP—the total read depth. 
</br>
There are others,
but they are mostly specific to the 1,000 Genomes Project (here, we will try to be as general as
possible). Your own dataset may have more annotations (or none of these).
At the sample level, there are only two annotations in this file: GT—genotype, and DP—the
per-sample read depth. You have the per-variant (total) read depth and the per-sample read
depth; be sure not to confuse both.

In [ ]:
pip install cyvcf2

In [18]:
# required package
from cyvcf2 import VCF

from cyvcf2 import VCF
v = VCF('genotypes.vcf.gz')
rec = next(v)
print('Variant Level information')
info = rec.INFO
for info in rec.INFO:
  print(info)
print('Sample Level information')
for fmt in rec.FORMAT:
  print(fmt)

Variant Level information
('AC', 1)
('AF', 0.000199681002413854)
('AN', 5008)
('NS', 2504)
('DP', 8012)
('ASN_AF', 0.0)
('AMR_AF', 0.0)
('SAS_AF', '0.0010')
('EUR_AF', 0.0)
('EAS_AF', '')
('AFR_AF', 0.0)
('SAN_AF', 0.0)
Sample Level information
GT
DP


**Now that we know what information is available, let’s inspect a single VCF record:**

We will start by retrieving the standard information: the chromosome, position, ID, reference base
(typically just one) and alternative bases (you can have more than one, but it’s not uncommon
as a first filtering approach to only accept a single ALT, for example, only accept biallelic
SNPs), quality (as you might expect, Phred-scaled), and filter status. Regarding the filter status,
remember that whatever the VCF file says, you may still want to apply extra filters (as in the
next recipe, Studying genome accessibility and filtering SNP data).
We then print the additional variant-level information (AC, AS, AF, AN, DP, and so on),
followed by the sample format (in this case, DP and GT). Finally, we count the number of
samples and inspect a single sample to check whether it was called for this variant. Also, the
reported alleles, heterozygosity, and phasing status (this dataset happens to be phased, which
is not that common) are included.

In [19]:
v = VCF('genotypes.vcf.gz')
samples = v.samples
print(len(samples))
variant = next(v)
print(variant.CHROM, variant.POS, variant.ID, variant.
REF, variant.ALT, variant.QUAL, variant.FILTER)
print(variant.INFO)
print(variant.FORMAT)
print(variant.is_snp)
str_alleles = variant.gt_bases[0]
alleles = variant.genotypes[0][0:2]
is_phased = variant.genotypes[0][2]
print(str_alleles, alleles, is_phased)
print(variant.format('DP')[0])

2504
22 16050075 None A ['G'] 100.0 None
['GT', 'DP']
True
A|A [0, 0] True
1


**Let’s check the type of variant and the number of nonbiallelic SNPs in a single pass:**

We will now use the now-common Python default dictionary. We find that this dataset has
INDELs, CNVs, and—of course—SNPs (roughly two-thirds being transitions with one-third
transversions). There is a residual number (79) of triallelic SNPs.

In [20]:
from collections import defaultdict
f = VCF('genotypes.vcf.gz')
my_type = defaultdict(int)
num_alts = defaultdict(int)
for variant in f:
  my_type[variant.var_type, variant.var_subtype] += 1
  if variant.var_type == 'snp':
    num_alts[len(variant.ALT)] += 1
print(my_type)

defaultdict(<class 'int'>, {('snp', 'ts'): 10054, ('snp', 'tv'): 5917, ('sv', 'CNV'): 2, ('indel', 'del'): 273, ('snp', 'unknown'): 79, ('indel', 'ins'): 127, ('indel', 'unknown'): 13, ('sv', 'DEL'): 6, ('sv', 'SVA'): 1})
